## Live Lecture Task

Yesterday we used a simple local workflow with SQLite - today, we'll work on
inserting the same RPG data into a more production-style PostgreSQL database
running on a server. We will use [psycopg](http://initd.org/psycopg/), a Python
library for connecting to PostgreSQL, and specifically we will install
[psycopg2-binary](https://pypi.org/project/psycopg2-binary/).

Once we get the data inserted, we will continue exploring querying as yesterday,
first answering the same questions and then going deeper. We'll also explore
some of the specific functions that are different in PostgreSQL than SQLite.

## Assignment

Reproduce (debugging as needed) the live lecture task of setting up and
inserting the RPG data into a PostgreSQL database, and add the code you write to
do so.

Then, set up a new table for the Titanic data (`titanic.csv`) - spend some time
thinking about the schema to make sure it is appropriate for the columns.
[Enumerated types](https://www.postgresql.org/docs/9.1/datatype-enum.html) may
be useful. Once it is set up, write a `insert_titanic.py` script that uses
`psycopg2` to connect to and upload the data from the csv, and add the file to
your repo. Then start writing PostgreSQL queries to explore the data!

In [353]:
import pandas as pd
import psycopg2
import sqlite3

In [354]:
# don't commit this 
dbname = 'dpcfouyb'
user = 'dpcfouyb'
password = 'kaIJ_GBlhtUhWMk96d3KmkP0U76T4kkA' 
host = 'salt.db.elephantsql.com'

pg_conn = psycopg2.connect(dbname=dbname, user=user,
                        password=password, host=host)

# connection object and cursor
pg_conn
pg_curs = pg_conn.cursor()


In [355]:
# extract: csv file
titanic_csv = 'titanic.csv'
df = pd.read_csv(titanic_csv)

df['Name'] = df['Name'].str.replace("'", "")

# create connection to blank sql 'titanic.sqlite3'
conn = sqlite3.connect('titanic.sqlite3')

# thus extract data from df to sql file
df.to_sql('titanic', conn, index=False, if_exists='replace') # Insert the values from the csv file into the table 'X'

In [356]:
# look at table 
curs = conn.cursor()
query = 'SELECT * FROM titanic LIMIT 20'
pd.read_sql(query, conn)

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500
5,0,3,Mr. James Moran,male,27.0,0,0,8.4583
6,0,1,Mr. Timothy J McCarthy,male,54.0,0,0,51.8625
7,0,3,Master. Gosta Leonard Palsson,male,2.0,3,1,21.0750
8,1,3,Mrs. Oscar W (Elisabeth Vilhelmina Berg) Johnson,female,27.0,0,2,11.1333
9,1,2,Mrs. Nicholas (Adele Achem) Nasser,female,14.0,1,0,30.0708


In [357]:
t_curs = conn.cursor()
query = 'SELECT COUNT(*) FROM titanic;'

t_curs.execute(query).fetchall()

[(887,)]

In [358]:
# our goal - an ETL/data pipeline from SQLite to Python
titanic = t_curs.execute('SELECT * FROM titanic;').fetchall()

In [359]:
# validate what we got
titanic[0]

(0, 3, 'Mr. Owen Harris Braund', 'male', 22.0, 1, 0, 7.25)

In [360]:
# look at data types
t_curs.execute('PRAGMA table_info(titanic);').fetchall()

[(0, 'Survived', 'INTEGER', 0, None, 0),
 (1, 'Pclass', 'INTEGER', 0, None, 0),
 (2, 'Name', 'TEXT', 0, None, 0),
 (3, 'Sex', 'TEXT', 0, None, 0),
 (4, 'Age', 'REAL', 0, None, 0),
 (5, 'Siblings/Spouses Aboard', 'INTEGER', 0, None, 0),
 (6, 'Parents/Children Aboard', 'INTEGER', 0, None, 0),
 (7, 'Fare', 'REAL', 0, None, 0)]

In [361]:
# extract done! next step, transform:
# we need the postgresql db to have a table
# with an appropriate schema


# drop table if exists first
pg_curs.execute("DROP TABLE IF EXISTS titanic")
pg_conn.commit()

# we need a serial primary key as it's what links
# all the tables together
create_titanic_table = """
    CREATE TABLE titanic (
        id SERIAL PRIMARY KEY, 
        Survived INT,
        Pclass INT,
        Name VARCHAR(100),
        Sex VARCHAR(10),
        Age REAL,
        Siblings_Spouses_Aboard INT,
        Parents_Children_Aboard INT,
        Fare REAL
);

"""

In [362]:
# create pg table
pg_curs.execute(create_titanic_table)
pg_conn.commit()

In [363]:
str(titanic[0])

"(0, 3, 'Mr. Owen Harris Braund', 'male', 22.0, 1, 0, 7.25)"

In [364]:
# transform (making the target ready to get data) done

# now we need to insert actual characters
# example first

example_insert = """
INSERT INTO titanic
(Survived, PClass, Name, Sex, Age, Siblings_Spouses_Aboard, Parents_Children_Aboard, Fare)
VALUES """ + str(titanic[0])

print(example_insert)


INSERT INTO titanic
(Survived, PClass, Name, Sex, Age, Siblings_Spouses_Aboard, Parents_Children_Aboard, Fare)
VALUES (0, 3, 'Mr. Owen Harris Braund', 'male', 22.0, 1, 0, 7.25)


In [367]:
# now do this for all characters

for row in titanic: # this refers to titanic in row 25
    insert_titanic = """
        INSERT INTO titanic
        (id, Survived, PClass, Name, Sex, Age, Siblings_Spouses_Aboard, Parents_Children_Aboard, Fare)
        VALUES """ + str(row[0]) + ';' 
    pg_curs.execute(insert_titanic)
    

SyntaxError: syntax error at or near "0"
LINE 4:         VALUES 0;
                       ^


In [ ]:
pg_curs.execute('SELECT * FROM titanic;')
pg_curs.fetchall()

In [ ]:
# we can see it from this cursor but not elephantsql.com
# we must commit

pg_curs.close()
pg_conn.commit()